# SPR 2026 - TF-IDF + CatBoost

**Notebook para submissão offline no Kaggle.**

---
**CONFIGURAÇÃO OFFLINE:**
1. No Kaggle, vá em Settings → Internet → **OFF**
2. Não requer datasets externos de modelos
---

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings('ignore')

SEED = 42
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
np.random.seed(SEED)
print('Bibliotecas carregadas!')

In [ ]:
# Carregar dados
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')

print(f'Train: {train.shape}')
print(f'Test: {test.shape}')

In [ ]:
# TF-IDF Vectorizer
tfidf = TfidfVectorizer(
    max_features=20000,
    ngram_range=(1, 3),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True
)

X_train = tfidf.fit_transform(train['report'])
X_test = tfidf.transform(test['report'])
y_train = train['target'].values

print(f'TF-IDF shape: {X_train.shape}')

In [ ]:
# CatBoost
model = CatBoostClassifier(
    iterations=500,
    depth=8,
    learning_rate=0.05,
    auto_class_weights='Balanced',
    random_seed=SEED,
    verbose=100,
    thread_count=-1
)

# Cross-validation
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1_macro')
print(f'CV F1-Macro: {scores.mean():.5f} (+/- {scores.std():.5f})')

# Treinar modelo final
model.fit(X_train, y_train)
print('Modelo treinado!')

In [ ]:
# Predições e submissão
predictions = model.predict(X_test)

submission = pd.DataFrame({
    'ID': test['ID'],
    'target': predictions.flatten()
})

submission.to_csv('submission.csv', index=False)

print('submission.csv criado!')
print(submission['target'].value_counts().sort_index())